In [114]:
# run me first!

from sklearn.linear_model import LogisticRegression
from timeit import default_timer as timer
from datetime import timedelta
import os
from sklearn import preprocessing
import numpy as np
from read_scripts import dict_2_arr 
from read_scripts import read_dataset 
from sklearn.utils.extmath import randomized_svd
from sklearn.decomposition import PCA

task = "remote_homology"

def fit_logistic(X, y):
    start = timer()
    clf = LogisticRegression(max_iter=5000)
    clf.fit(X, y)
    end = timer()
    print(f"fit time: ", timedelta(seconds=end-start))
    
    train_score = clf.score(X, y)
    print(f"model train score: ", train_score)
    
    # when run will play a ping sound!
    os.system("printf '\a'")
    os.system("printf '\a'")
    os.system("printf '\a'")
    print("Done!")
    
    return clf



In [122]:
def train_test_over_corpus(corpus, ys, model_name):

    all_score = {}

    # get out training slice
    X_train = corpus[:len(ys[0])]
    y_train = ys[0]

    # scale
    scaler = preprocessing.StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train)

    # fit!
    clf = fit_logistic(X_train, y_train)

    # record train score
    train_score = clf.score(X_train, y_train)
    print(f"{model_name} train score: ", train_score)

    all_score["train"]  = train_score

    # get slices for remaing splits and score
    remain_splits = ['valid', 'test_fold_holdout', 'test_superfamily_holdout', 'test_family_holdout']
    end = len(ys[0])
    for i in range(len(remain_splits)):
        split = remain_splits[i]
        start = end
        end = len(ys[i+1])+start

        X = corpus[start:end]
        y = ys[i+1]

        X = scaler.transform(X)

        test_score = clf.score(X, y)

        all_score[split]  = test_score

        print(f"{model_name} {split} score: ", test_score)

    return all_score, clf

In [130]:
def get_ys():
    ys = []
    # read all splits
    for split in ['train', 'valid', 'test_fold_holdout', 'test_superfamily_holdout', 'test_family_holdout']:
        y_dict = read_dataset('label', 'remote_homology', split)
        X, y = dict_2_arr(read_dataset('elmo', task, split), y_dict)
        ys.append(y)
        
    return ys

In [10]:
def get_single_model_normalized_corpus(model):
    ys = []
    Xs = []

    # read all splits
    for split in ['train', 'valid', 'test_fold_holdout', 'test_superfamily_holdout', 'test_family_holdout']:

        y_dict = read_dataset('label', task, split)
        X, y = dict_2_arr(read_dataset(model, task, split), y_dict)
        
        ys.append(y)
        Xs.append(X)  
    
    # concat all splits
    corpus = np.concatenate(Xs, axis=0)
    
    return corpus, ys

In [11]:
X_e, y = get_single_model_normalized_corpus('elmo')
X_t, y = get_single_model_normalized_corpus('transformer')
X_u, y = get_single_model_normalized_corpus('unirep')

In [111]:
lr = 0.0000001
max_iter = 6000
reg = 1

def one_to_N(embs_list, meta_dim):

    Ms = []
    Vs = []
    for Vi in embs_list:
        Mi = np.random.rand(Vi.shape[1], meta_dim)
        Ms.append(Mi)
        Vs.append(np.transpose(Vi))
    
    n_examples = Vs[0].shape[1]    
    Vstar = np.random.rand(meta_dim, n_examples)
    
    r = len(Ms)
    itr = 0
    prev_norm = 0
    stop = False
    under_count = 0
    threshold = 0.0001
    
    while itr < max_iter and not stop:
        wn = Vstar
        term1 = np.zeros((meta_dim,n_examples))
        term2 = np.zeros((meta_dim,n_examples))
        for i in range(r):

            wi = Vs[i]
        
            wnT = np.transpose(wn)
            Mi = Ms[i]
            # M_i for next itr
            t1 = Mi.dot(wn.dot(wnT)) 
            t2 = reg*Mi
            t3 = wi.dot(wnT)
            Mip1 = Mi - lr*(t1 + t2 - t3)
            Ms[i] = Mip1
            
            # w for next itr
            MiT = np.transpose(Mi)
            res = MiT.dot(Mi.dot(wn)) 
            term1 = term1 + res
            term2 = term2 + MiT.dot(wi)


        Vstar = wn - lr*(term1 - term2)
        
        # compute cost over epoch
        cost = 0
        proj_norm = 0
        for i in range(r):
            Mi = Ms[i]
            wi = Vs[i]
            wi_hat = Mi.dot(Vstar)
            cost += np.linalg.norm(wi_hat - wi)**2 + reg*(np.linalg.norm(Mi)**2)
            proj_norm += np.linalg.norm(wi_hat - wi)
        
        print(f"Iteration {itr}, cost {cost}, proj_norm {proj_norm}")
        
        # stopping conditons
        if abs(prev_norm - proj_norm) < threshold:
            under_count += 1
        else:
            under_count
        prev_norm = proj_norm   
        if(under_count > 10):
            print(f"Difference in proj under {threshold} for 10 iterations. Stopping.")
            stop = True
        
        itr += 1
        
    print(f"Max iterations {max_iter} reached. Stopping.")
    return Vstar

In [112]:
embed = one_to_N([X_e, X_t, X_u], 500)

Iteration 0, cost 543042997447.0309, proj_norm 1252816.2155140226
Iteration 1, cost 339836717720.596, proj_norm 991072.8548514917
Iteration 2, cost 222765001815.28442, proj_norm 802405.8446294946
Iteration 3, cost 150315386839.51636, proj_norm 659132.1670775567
Iteration 4, cost 103376724198.76898, proj_norm 546616.0534657898
Iteration 5, cost 72021352897.98276, proj_norm 456248.9690038079
Iteration 6, cost 50633014180.16884, proj_norm 382550.2424515814
Iteration 7, cost 35830058283.279144, proj_norm 321807.22278443375
Iteration 8, cost 25480020391.05731, proj_norm 271376.66956478416
Iteration 9, cost 18191199347.31411, proj_norm 229300.0227274531
Iteration 10, cost 13031946455.283197, proj_norm 194079.28498011624
Iteration 11, cost 9366794997.161175, proj_norm 164540.02924873668
Iteration 12, cost 6756311650.873425, proj_norm 139743.94066056976
Iteration 13, cost 4893567528.185423, proj_norm 118930.64840125077
Iteration 14, cost 3562620758.6579523, proj_norm 101477.4724252751
Iteratio

Iteration 122, cost 201593529.10040936, proj_norm 24151.989388293165
Iteration 123, cost 201524629.07716036, proj_norm 24147.86808707143
Iteration 124, cost 201455760.74578565, proj_norm 24143.747978734904
Iteration 125, cost 201386924.08693418, proj_norm 24139.629062736058
Iteration 126, cost 201318119.0812686, proj_norm 24135.51133852764
Iteration 127, cost 201249345.7094673, proj_norm 24131.39480556279
Iteration 128, cost 201180603.9522239, proj_norm 24127.279463295006
Iteration 129, cost 201111893.79024687, proj_norm 24123.165311178134
Iteration 130, cost 201043215.2042593, proj_norm 24119.052348666344
Iteration 131, cost 200974568.17499927, proj_norm 24114.940575214154
Iteration 132, cost 200905952.68322143, proj_norm 24110.82999027652
Iteration 133, cost 200837368.7096933, proj_norm 24106.720593308644
Iteration 134, cost 200768816.2351988, proj_norm 24102.612383766136
Iteration 135, cost 200700295.2405362, proj_norm 24098.505361104937
Iteration 136, cost 200631805.70651817, proj_

Iteration 242, cost 193546492.73807478, proj_norm 23665.799503908987
Iteration 243, cost 193481261.68822998, proj_norm 23661.817535924692
Iteration 244, cost 193416060.1118152, proj_norm 23657.836697554652
Iteration 245, cost 193350887.99119046, proj_norm 23653.856988291616
Iteration 246, cost 193285745.30872843, proj_norm 23649.87840762861
Iteration 247, cost 193220632.04681638, proj_norm 23645.90095505904
Iteration 248, cost 193155548.18785387, proj_norm 23641.924630076574
Iteration 249, cost 193090493.71425462, proj_norm 23637.949432175228
Iteration 250, cost 193025468.60844553, proj_norm 23633.975360849334
Iteration 251, cost 192960472.85286665, proj_norm 23630.002415593517
Iteration 252, cost 192895506.4299719, proj_norm 23626.03059590276
Iteration 253, cost 192830569.3222281, proj_norm 23622.05990127233
Iteration 254, cost 192765661.51211604, proj_norm 23618.09033119784
Iteration 255, cost 192700782.9821294, proj_norm 23614.121885175206
Iteration 256, cost 192635933.71477467, pro

Iteration 363, cost 185862544.70475128, proj_norm 23192.03895914474
Iteration 364, cost 185800755.6451048, proj_norm 23188.190081618588
Iteration 365, cost 185738994.0186676, proj_norm 23184.34227457548
Iteration 366, cost 185677259.80934113, proj_norm 23180.495537545
Iteration 367, cost 185615553.0010392, proj_norm 23176.649870057056
Iteration 368, cost 185553873.57768756, proj_norm 23172.805271641817
Iteration 369, cost 185492221.52322322, proj_norm 23168.961741829742
Iteration 370, cost 185430596.82159695, proj_norm 23165.119280151644
Iteration 371, cost 185368999.45676994, proj_norm 23161.27788613853
Iteration 372, cost 185307429.41271603, proj_norm 23157.437559321766
Iteration 373, cost 185245886.67342108, proj_norm 23153.598299232985
Iteration 374, cost 185184371.2228825, proj_norm 23149.76010540409
Iteration 375, cost 185122883.04510987, proj_norm 23145.922977367292
Iteration 376, cost 185061422.12412524, proj_norm 23142.08691465511
Iteration 377, cost 184999988.44396138, proj_n

Iteration 483, cost 178639306.44947127, proj_norm 22737.687139783717
Iteration 484, cost 178580697.72586724, proj_norm 22733.963441251362
Iteration 485, cost 178522114.58664164, proj_norm 22730.240758763975
Iteration 486, cost 178463557.01706445, proj_norm 22726.519091884336
Iteration 487, cost 178405025.00241864, proj_norm 22722.79844017562
Iteration 488, cost 178346518.52799672, proj_norm 22719.0788032012
Iteration 489, cost 178288037.57910156, proj_norm 22715.360180524687
Iteration 490, cost 178229582.14104813, proj_norm 22711.642571710025
Iteration 491, cost 178171152.1991607, proj_norm 22707.925976321345
Iteration 492, cost 178112747.73877567, proj_norm 22704.210393923124
Iteration 493, cost 178054368.74523968, proj_norm 22700.495824080055
Iteration 494, cost 177996015.20390904, proj_norm 22696.78226635706
Iteration 495, cost 177937687.10015264, proj_norm 22693.069720319403
Iteration 496, cost 177879384.4193483, proj_norm 22689.358185532525
Iteration 497, cost 177821107.1468861, p

...

Iteration 5496, cost 46165571.754490495, proj_norm 11587.45283945672
Iteration 5497, cost 46154929.779612646, proj_norm 11586.125530537643
Iteration 5498, cost 46144290.775186054, proj_norm 11584.798439683493
Iteration 5499, cost 46133654.740216196, proj_norm 11583.471566845179
Iteration 5500, cost 46123021.67370878, proj_norm 11582.144911973613
Iteration 5501, cost 46112391.57467006, proj_norm 11580.818475019732
Iteration 5502, cost 46101764.44210646, proj_norm 11579.492255934463
Iteration 5503, cost 46091140.275025204, proj_norm 11578.166254668799
Iteration 5504, cost 46080519.0724334, proj_norm 11576.840471173678
Iteration 5505, cost 46069900.83333901, proj_norm 11575.514905400121
Iteration 5506, cost 46059285.55675024, proj_norm 11574.189557299133
Iteration 5507, cost 46048673.241675586, proj_norm 11572.864426821727
Iteration 5508, cost 46038063.88712418, proj_norm 11571.539513918959
Iteration 5509, cost 46027457.49210532, proj_norm 11570.21481854187
Iteration 5510, cost 46016854.0

Iteration 5615, cost 44919763.11573694, proj_norm 11431.02079103082
Iteration 5616, cost 44909467.75067157, proj_norm 11429.719091360566
Iteration 5617, cost 44899175.24048853, proj_norm 11428.417604021019
Iteration 5618, cost 44888885.58423989, proj_norm 11427.11632896489
Iteration 5619, cost 44878598.78097839, proj_norm 11425.815266144931
Iteration 5620, cost 44868314.82975704, proj_norm 11424.514415513908
Iteration 5621, cost 44858033.72962904, proj_norm 11423.213777024568
Iteration 5622, cost 44847755.47964835, proj_norm 11421.913350629718
Iteration 5623, cost 44837480.07886921, proj_norm 11420.613136282176
Iteration 5624, cost 44827207.52634587, proj_norm 11419.313133934724
Iteration 5625, cost 44816937.82113338, proj_norm 11418.013343540213
Iteration 5626, cost 44806670.96228705, proj_norm 11416.713765051494
Iteration 5627, cost 44796406.94886248, proj_norm 11415.414398421417
Iteration 5628, cost 44786145.77991575, proj_norm 11414.115243602864
Iteration 5629, cost 44775887.454503

Iteration 5734, cost 43714402.28877931, proj_norm 11277.596477902563
Iteration 5735, cost 43704440.10371265, proj_norm 11276.319717740449
Iteration 5736, cost 43694480.66337928, proj_norm 11275.043164387145
Iteration 5737, cost 43684523.96687572, proj_norm 11273.766817797128
Iteration 5738, cost 43674570.01329867, proj_norm 11272.490677924867
Iteration 5739, cost 43664618.801745415, proj_norm 11271.214744724863
Iteration 5740, cost 43654670.331313506, proj_norm 11269.939018151632
Iteration 5741, cost 43644724.601100765, proj_norm 11268.663498159687
Iteration 5742, cost 43634781.610205576, proj_norm 11267.388184703581
Iteration 5743, cost 43624841.35772656, proj_norm 11266.113077737864
Iteration 5744, cost 43614903.842762545, proj_norm 11264.83817721709
Iteration 5745, cost 43604969.064413115, proj_norm 11263.563483095862
Iteration 5746, cost 43595037.02177773, proj_norm 11262.288995328752
Iteration 5747, cost 43585107.71395672, proj_norm 11261.014713870394
Iteration 5748, cost 43575181

Iteration 5853, cost 42547928.4686366, proj_norm 11127.10166787892
Iteration 5854, cost 42538286.66060155, proj_norm 11125.849202142657
Iteration 5855, cost 42528647.49226688, proj_norm 11124.596937895416
Iteration 5856, cost 42519010.96277079, proj_norm 11123.34487509332
Iteration 5857, cost 42509377.071251854, proj_norm 11122.093013692507
Iteration 5858, cost 42499745.81684895, proj_norm 11120.841353649124
Iteration 5859, cost 42490117.19870139, proj_norm 11119.589894919343
Iteration 5860, cost 42480491.215948686, proj_norm 11118.338637459332
Iteration 5861, cost 42470867.86773087, proj_norm 11117.0875812253
Iteration 5862, cost 42461247.153188184, proj_norm 11115.836726173444
Iteration 5863, cost 42451629.071461104, proj_norm 11114.586072259972
Iteration 5864, cost 42442013.62169077, proj_norm 11113.335619441135
Iteration 5865, cost 42432400.803018324, proj_norm 11112.085367673164
Iteration 5866, cost 42422790.61458546, proj_norm 11110.83531691233
Iteration 5867, cost 42413183.05553

Iteration 5972, cost 41418853.35688062, proj_norm 10979.460992513086
Iteration 5973, cost 41409519.714762874, proj_norm 10978.232199594042
Iteration 5974, cost 41400188.61214447, proj_norm 10977.003603036666
Iteration 5975, cost 41390860.04820317, proj_norm 10975.775202798668
Iteration 5976, cost 41381534.0221168, proj_norm 10974.54699883774
Iteration 5977, cost 41372210.533063725, proj_norm 10973.31899111162
Iteration 5978, cost 41362889.58022258, proj_norm 10972.091179578041
Iteration 5979, cost 41353571.16277239, proj_norm 10970.86356419477
Iteration 5980, cost 41344255.27989252, proj_norm 10969.636144919577
Iteration 5981, cost 41334941.930762455, proj_norm 10968.40892171023
Iteration 5982, cost 41325631.11456211, proj_norm 10967.181894524518
Iteration 5983, cost 41316322.830471866, proj_norm 10965.955063320262
Iteration 5984, cost 41307017.07767217, proj_norm 10964.728428055263
Iteration 5985, cost 41297713.85534404, proj_norm 10963.50198868737
Iteration 5986, cost 41288413.162668

In [113]:
embed.shape

(500, 16281)

In [131]:
embed_final = np.transpose(embed)
embed_final.shape

(16281, 500)

In [132]:
ys = get_ys()

In [133]:
score, clf = train_test_over_corpus(embed_final, ys, "1TON dim 500")

fit time:  0:06:37.041109
model train score:  0.8416903697683868
Done!
1TON dim 500 train score:  0.8416903697683868
1TON dim 500 valid score:  0.004087193460490463
1TON dim 500 test_fold_holdout score:  0.004178272980501393
1TON dim 500 test_superfamily_holdout score:  0.00878594249201278
1TON dim 500 test_family_holdout score:  0.02122641509433962
